In [1]:
def process_intcode(data, input_values):
    input_values = (i for i in input_values)
    i = 0
    while True:
        initial = str(data[i]).rjust(5, '0')
        a, b, c, *opcode = initial
        opcode = int(''.join(opcode))
        assert opcode in (1, 2, 3, 4, 5, 6, 7, 8, 99), opcode
        if opcode == 99:
            return
        p1 = i+1 if c == '1' else data[i+1]
        p2 = i+2 if b == '1' else data[i+2]
        try:
            p3 = i+3 if a == '1' else data[i+3]
        except IndexError:
            p3 = None
        if opcode == 1:
            data[p3] = data[p1] + data[p2]
            i += 4
        elif opcode == 2:
            data[p3] = data[p1] * data[p2]
            i += 4
        elif opcode == 3:
            data[p1] = next(input_values)
            i += 2
        elif opcode == 4:
            return data[p1]
            i += 2
        elif opcode == 5:
            if data[p1] != 0:
                i = data[p2]
            else:
                i += 3
        elif opcode == 6:
            if data[p1] == 0:
                i = data[p2]
            else:
                i += 3
        elif opcode == 7:
            data[p3] = 1 if data[p1] < data[p2] else 0
            i += 4
        elif opcode == 8:
            data[p3] = 1 if data[p1] == data[p2] else 0
            i += 4

In [2]:
from itertools import permutations

def calc_signal(intcode, settings):
    this_intcode = intcode.copy()
    output = 0
    for setting in settings:
        input_values = [setting, output]
        output = process_intcode(this_intcode, input_values)
    return output

def calc_max_signal(intcode):
    signals = {
        settings: calc_signal(intcode, settings)
        for settings in permutations(range(5), 5)
    }
    max_signal_settings = max(signals, key=signals.get)
    return (max_signal_settings, signals[max_signal_settings])

sample_intcode = [3,15,3,16,1002,16,10,16,1,16,15,15,4,15,99,0,0]
assert (r := calc_max_signal(sample_intcode)) == ((4,3,2,1,0), 43210), r
sample_intcode = [3,23,3,24,1002,24,10,24,1002,23,-1,23,101,5,23,23,1,24,23,23,4,23,99,0,0]
assert (r := calc_max_signal(sample_intcode)) == ((0,1,2,3,4), 54321), r
sample_intcode = [3,31,3,32,1002,32,10,32,1001,31,-2,31,1007,31,0,33,1002,33,7,33,1,33,31,31,1,32,31,31,4,31,99,0,0,0]
assert (r := calc_max_signal(sample_intcode)) == ((1,0,4,3,2), 65210), r

In [3]:
with open('input') as f:
    intcode = [int(d) for d in f.read().split(',')]

In [4]:
print("Part 1:")
calc_max_signal(intcode)[1]

Part 1:


65464

In [5]:
class Amplifier:
    def __init__(self, intcode, setting, initial_input=None, name=None):
        self.name = name
        self.intcode = intcode.copy()
        self.setting = setting
        if initial_input is None:
            self._input_values = [setting]
        else:
            self._input_values = [setting, initial_input]
        self.next_amplifier = None
        self._last_output_value = None
        self.i = 0
        self.halted = False
    
    @property
    def input(self):
        value = self._input_values.pop(0)
        return value
    
    @input.setter
    def input(self, value):
        self._input_values.append(value)
    
    @property
    def output(self):
        return self._last_output_value
    
    @output.setter
    def output(self, value):
        self._last_output_value = value
        self.next_amplifier.input = value
            
    def compute(self):
        while True:
            initial = str(self.intcode[self.i]).rjust(5, '0')
            a, b, c, *opcode = initial
            opcode = int(''.join(opcode))
            assert opcode in (1, 2, 3, 4, 5, 6, 7, 8, 99), opcode
            if opcode == 99:
                self.halted = True
                if not self.next_amplifier.halted:
                    return self.next_amplifier.compute()
                return
            p1 = self.i+1 if c == '1' else self.intcode[self.i+1]
            p2 = self.i+2 if b == '1' else self.intcode[self.i+2]
            try:
                p3 = self.i+3 if a == '1' else self.intcode[self.i+3]
            except IndexError:
                p3 = None
            if opcode == 1:
                self.intcode[p3] = self.intcode[p1] + self.intcode[p2]
                self.i += 4
            elif opcode == 2:
                self.intcode[p3] = self.intcode[p1] * self.intcode[p2]
                self.i += 4
            elif opcode == 3:
                try:
                    self.intcode[p1] = self.input
                    self.i += 2
                except IndexError:
                    return self.next_amplifier.compute()
            elif opcode == 4:
                self.output = self.intcode[p1]
                self.i += 2
                return self.next_amplifier.compute()
            elif opcode == 5:
                if self.intcode[p1] != 0:
                    self.i = self.intcode[p2]
                else:
                    self.i += 3
            elif opcode == 6:
                if self.intcode[p1] == 0:
                    self.i = self.intcode[p2]
                else:
                    self.i += 3
            elif opcode == 7:
                self.intcode[p3] = 1 if self.intcode[p1] < self.intcode[p2] else 0
                self.i += 4
            elif opcode == 8:
                self.intcode[p3] = 1 if self.intcode[p1] == self.intcode[p2] else 0
                self.i += 4
    
test_intcode = [3,26,1001,26,-4,26,3,27,1002,27,2,27,1,27,26,27,4,27,1001,28,-1,28,1005,28,6,99,0,0,5]
A = Amplifier(name='A', intcode=test_intcode, setting=9, initial_input=0)
B = Amplifier(name='B', intcode=test_intcode, setting=8)
C = Amplifier(name='C', intcode=test_intcode, setting=7)
D = Amplifier(name='D', intcode=test_intcode, setting=6)
E = Amplifier(name='E', intcode=test_intcode, setting=5)

A.next_amplifier = B
B.next_amplifier = C
C.next_amplifier = D
D.next_amplifier = E
E.next_amplifier = A

A.compute()
assert E.output == 139629729

In [6]:
def calc_signal_2(intcode, settings):
    amp_settings = [[s, None] for s in settings]
    amp_settings[0][1] = 0
    amps = [
        Amplifier(intcode=intcode, setting=s, initial_input=ii)
        for s, ii in amp_settings
    ]
    for i, amp in enumerate(amps):
        amp.next_amplifier = amps[(i+1) % len(amps)]

    amps[0].compute()
    return amps[-1].output

assert calc_signal_2(test_intcode, (9, 8, 7, 6, 5)) == 139629729

In [7]:
def calc_max_signal_2(intcode):
    signals = {
        settings: calc_signal_2(intcode, settings)
        for settings in permutations(range(5, 10), 5)
    }
    max_signal_settings = max(signals, key=signals.get)
    return (max_signal_settings, signals[max_signal_settings])

assert (r := calc_max_signal_2(test_intcode)) == ((9,8,7,6,5), 139629729), r

test_intcode = [3,52,1001,52,-5,52,3,53,1,52,56,54,1007,54,5,55,1005,55,26,1001,54,-5,54,1105,1,12,1,53,54,53,1008,54,0,55,1001,55,1,55,2,53,55,53,4,53,1001,56,-1,56,1005,56,6,99,0,0,0,0,10]
assert (r := calc_max_signal_2(test_intcode)) == ((9,7,8,5,6), 18216), r

In [8]:
print("Part 2:")
calc_max_signal_2(intcode)[1]

Part 2:


1518124